In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixregV4_norm.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixregV4_norm.csv
day8_features_matrixregV4_norm.csv
day9_features_matrixregV4_norm.csv
test_features_matrixregV4_norm.csv


In [4]:
## load category col
file_format = '{}_category_discrete_scale10_new.csv'
df_dict_cat = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict_cat[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_category_discrete_scale10_new.csv
day8_category_discrete_scale10_new.csv
day9_category_discrete_scale10_new.csv
test_category_discrete_scale10_new.csv


In [6]:
# replace/add new discrete cols
for file in ['day7', 'day8', 'day9','test']: 
#     df_dict[file]['ip'] = df_dict_cat[file]['ip']
#     for col in ['app', 'device', 'os', 'channel']:
    for col in ['app', 'device', 'os', 'channel']:
#         df_dict[file].loc[:,col] = df_dict_cat[file][col].values
        df_dict[file][col+'_extra'] = df_dict_cat[file][col]

In [7]:
df_dict['day7'].head(100)

,ip_day_hour_count,ip_os_day_hour_count,ip_app_day_hour_count,ip_app_os_day_hour_count,app_day_hour_count,ip_device_os_count,ip_app_device_os_count,ip_device_os_mean,ip_app_device_os_mean,ip_app_device_mean,...,app,channel,device,os,hour,is_attributed,app_extra,device_extra,os_extra,channel_extra
0,11,9,3,3,430196,31,11,0.000000,0.000000,0.000000,...,12,245,1,17,4,0,1,1,1,1
1,71,9,8,3,430196,121,21,0.013333,0.000000,0.000000,...,12,245,1,18,4,0,1,1,1,1
2,11,9,3,3,312628,31,9,0.000000,0.000000,0.000000,...,18,317,1,17,4,0,1,1,1,1
3,655,129,222,124,327851,1008,260,0.000000,0.000000,0.000199,...,15,245,1,10,4,0,1,1,1,1
4,55,55,11,11,754715,59,12,0.000000,0.000000,0.000000,...,3,480,1,8,4,0,1,1,1,1
5,185,52,8,3,66351,2684,52,0.000853,0.000000,0.000000,...,11,173,1,19,4,0,1,1,1,1
6,170,3,41,3,754715,406,74,0.000000,0.000000,0.000000,...,3,280,1,15,4,0,1,1,1,1
7,101,28,13,3,495324,131,11,0.000000,0.000000,0.000000,...,2,219,1,6,4,0,1,1,1,1
8,107,72,31,24,754715,478,108,0.000000,0.000000,0.000000,...,3,452,1,13,4,0,1,1,1,1
9,97,8,6,1,495324,616,69,0.003788,0.000000,0.000000,...,2,122,1,13,4,0,1,1,1,1


# define parameter

In [8]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour', 'app_extra', 'channel_extra', 'device_extra', 'os_extra']
feature_cols = list(df_dict['test'].columns.values)

In [9]:
df_dict['day7'].columns.values

array(['ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'app_device_os_time2nextclick',
       'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick',
       'app_device_os_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'app_device_os_countfromfuture', 'ip_device_os_countfrompast',
       'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast',
       'app_device_os_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
  

In [10]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['app_device_os_mean',
             'app_device_os_time2nextclick', 
             'app_device_os_time2previousclick',
             'app_device_os_countfromfuture',
             'app_device_os_countfrompast', 
             'app_device_os_lasttimediff', 
             'app_device_os_regression'])
# extra = set(['ip_app_device_mean',
#              'ip_app_device_time2nextclick', 
#              'ip_app_device_time2previousclick',
#              'ip_app_device_countfromfuture',
#              'ip_app_device_countfrompast', 
#              'ip_app_device_lasttimediff', 
#              'ip_app_device_regression'])
feature_cols = list(set(feature_cols) - extra)

In [11]:
feature_cols

['ip_device_os_time2previousclick',
 'matrixFact_user_ip_item_appdeviceos',
 'ip_app_device_os_regression',
 'ip_day_hour_count',
 'ip_app_device_os_countfromfuture',
 'device_extra',
 'ip_device_os_countfromfuture',
 'os_extra',
 'ip_device_os_lasttimediff',
 'ip_app_device_os_mean',
 'ip_app_device_os_lasttimediff',
 'ip_app_os_day_hour_count',
 'ip_app_day_hour_count',
 'ip_app_device_time2previousclick',
 'app_day_hour_count',
 'channel',
 'channel_extra',
 'ip_app_device_regression',
 'ip_app_device_time2nextclick',
 'device',
 'app_extra',
 'ip_device_os_count',
 'ip_app_device_os_time2previousclick',
 'ip_app_device_countfrompast',
 'ip_app_device_os_countfrompast',
 'ip_app_device_mean',
 'ip_device_os_countfrompast',
 'matrixFact_user_iposdeviceapp_item_app',
 'hour',
 'app',
 'ip_app_device_os_count',
 'os',
 'ip_app_device_lasttimediff',
 'ip_os_day_hour_count',
 'ip_device_os_mean',
 'ip_device_os_regression',
 'ip_app_device_countfromfuture',
 'ip_device_os_time2nextclick'

In [12]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [13]:
for day in day_list:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_predtest_matrixregV4_norm_discrete_allnoip_retain_ori.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    df_all = pd.concat([trainset, valset])
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_{}{}_val_{}_matrixregV4_norm_discrete_allnoip_retain_ori.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)

    

concat_day8day9_val_day7_predtest_matrixregV4_norm_discrete_allnoip_retain_ori.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.97993
[20]	valid_0's auc: 0.981266
[30]	valid_0's auc: 0.982412
[40]	valid_0's auc: 0.983195
[50]	valid_0's auc: 0.983774
[60]	valid_0's auc: 0.984091
[70]	valid_0's auc: 0.984253
[80]	valid_0's auc: 0.984297
[90]	valid_0's auc: 0.984444
[100]	valid_0's auc: 0.984486
[110]	valid_0's auc: 0.984493
[120]	valid_0's auc: 0.984497
[130]	valid_0's auc: 0.984491
[140]	valid_0's auc: 0.984462
[150]	valid_0's auc: 0.984474
[160]	valid_0's auc: 0.984453
[170]	valid_0's auc: 0.984416
[180]	valid_0's auc: 0.984364
[190]	valid_0's auc: 0.984349
[200]	valid_0's auc: 0.984325
Early stopping, best iteration is:
[105]	valid_0's auc: 0.984514
start training
[10]	valid_0's auc: 0.981263
[20]	valid_0's auc: 0.982717
[30]	valid_0's auc: 0.984259
[40]	valid_0's auc: 0.985439
[50]	valid_0's auc: 0.986457
[60]	valid_0's auc: 0.98724
[70]	valid_0's auc: 0.987887
[80]	valid_0's auc: 0.988422
[90]	valid_0's auc: 0.988825
[100]	v

KeyboardInterrupt: 

In [ ]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

In [ ]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

# Average

In [24]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixregV4_norm_discrete_ip.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixregV4_norm_discrete_ip.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixregV4_norm_discrete_ip.npy')

pred = (day7 + day8 + day9)/3

In [25]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixregV4_norm_discrete_ip.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
